In [1]:
!nvidia-smi

Fri Apr 26 07:51:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:C7:00.0 Off |                    0 |
| N/A   38C    P0             119W / 700W |  10147MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [28]:
# fix numpy in colab
import numpy
from IPython.display import clear_output


!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --cache-dir $TMP_DIR --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/hice1/abambhaniya3/pace-ice-home-data/mixtral-offloading-residency-info/notebooks/Mixtral-8x7B-Instruct-v0.1-offloading-demo1


In [10]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
from datasets import load_dataset

import time
import gc
from src.build_model import OffloadConfig, QuantConfig, build_model

In [4]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [5]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo2"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 6
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


# del model

gc.collect
torch.cuda.empty_cache()


## DEFAULT with 4 Experts on GPU

In [6]:
if 'model' in locals():
    del model
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="TOP-K",
    routing_threshold=0.00
)

/home/hice1/abambhaniya3/scratch/moe_offload/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  6.50it/s]


In [7]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
# while True:
user_input = "Where is Georgia Tech? What is the name of its mascot?"

user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(model.device)

if past_key_values is None:
  attention_mask = torch.ones_like(input_ids)
else:
  seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
  attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=model.device)



# sequence = result["sequences"]
# past_key_values = result["past_key_values"]

In [8]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids,
  attention_mask=attention_mask,
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=100,
  max_new_tokens=100,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  output_hidden_states=False,
  output_router_logits=True,
  output_scores=True, 
)
latency = time.time() - start_time
total_experts_saved = 0
for i in result['router_logits'][-32:]:
    total_experts_saved += i[1]
total_experts_saved 

Num_tokens = result['sequences'].shape[1] - input_ids.shape[1]
print(f"Total Latency :{latency} sec, Throughput:{Num_tokens/latency}, Expert Loads saved:{total_experts_saved} ")

Georgia Tech, officially known as the Georgia Institute of Technology, is located in Atlanta, Georgia, in the United States. It is a public research university known for its strong programs in engineering, computing, and the sciences.

The mascot of Georgia Tech is a yellow-colored, costumed bird named "Buzz." Buzz is a yellow-beaked, furry bird with oversized wings and a chest full of Georgia Tech logo. Buzz often wears a bow tie
Total Latency :30.456985235214233 sec, Throughput:3.283319055635895, Expert Loads saved:0 


In [9]:
result.keys()

odict_keys(['sequences', 'scores', 'past_key_values', 'router_logits'])

## Calculate Perplexity

In [29]:

# test = load_dataset("allenai/c4", "en", split="validation", cache_dir='~/scratch/', streaming=True)
# all_text = ""

# for row in list(test.take(100)):
#      all_text += "\n\n" + row['text']

# encodings = tokenizer(all_text, return_tensors="pt")


In [12]:

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test[:100]")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")



In [31]:
# ! export HF_DATASETS_CACHE="~/scratch/llama3"

In [32]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("/home/hice1/abambhaniya3/scratch/models--meta-llama--Meta-Llama-3-8B-Instruct/LLaMA3-8B-Instrct").to(device)

In [13]:
import torch
from tqdm import tqdm

max_length = 4096
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

 38%|███████████████████████████████████████████████████▉                                                                                   | 5/13 [00:27<00:43,  5.46s/it]


In [14]:
ppl

## Mixtral Thresholding 0.25 has 7.7675

tensor(5.8643, device='cuda:0')

In [26]:
input_ids

tensor([[ 3687,   297,   272,  ...,   842, 28705,    13]], device='cuda:0')

In [27]:
target_ids, 

(tensor([[ -100,  -100,  -100,  ...,   842, 28705,    13]], device='cuda:0'),)

In [19]:
outputs

MoeCausalLMOutputWithPast(loss=tensor(1.6049, device='cuda:0'), aux_loss=None, logits=tensor([[[-1.4131, -1.4170,  8.7656,  ..., -2.0566, -0.2167,  2.0352],
         [ 0.5425,  0.5366,  7.6133,  ...,  1.2744, -0.9961,  0.7920],
         [ 0.1855,  0.1823,  8.7109,  ..., -0.6143,  2.9941,  2.4551],
         ...,
         [-2.3887, -2.3828, 12.3984,  ...,  0.1981, -3.4844,  1.2871],
         [-5.2695, -5.2734,  6.9141,  ..., -3.6992, -3.3848, -0.1780],
         [-5.1602, -5.1523,  9.5938,  ..., -5.3867, -5.3711, -0.3459]]],
       device='cuda:0'), past_key_values=((tensor([[[[ 1.5488e+00, -7.6416e-01, -3.8770e+00,  ...,  1.7705e+00,
            1.7578e+00, -2.9688e-01],
          [ 5.3008e+00, -3.9023e+00, -2.2129e+00,  ...,  1.7305e+00,
            1.0283e+00, -1.8799e+00],
          [ 2.4980e+00, -2.6113e+00,  7.3242e-03,  ...,  8.3594e-01,
           -2.9663e-01, -3.2148e+00],
          ...,
          [ 3.8340e+00, -2.6445e+00,  2.6016e+00,  ...,  1.6104e+00,
            1.2881e+00, 

In [ ]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(nlls, fp)

In [ ]:
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [ ]:
"\n\n".join(test["text"])


In [24]:
len(range(0, seq_len, stride)), len(nlls)

(81, 74)

In [4]:
import pickle , os
import torch

for file in os.listdir():
    if "nlls" in file:
        with open(file, 'rb') as topk_file:
            topk_nnls = pickle.load(topk_file)
    
        print(file, torch.exp(torch.stack(topk_nnls).mean()) )

nlls_wikitext_1024_512 tensor(4.7940, device='cuda:0')
nlls_wikitext_TOP-K_0_1024_512 tensor(4.7940, device='cuda:0')
nlls_wikitext_THRESHOLDING_0.05_1024_512 tensor(4.7940, device='cuda:0')
nlls_wikitext_THRESHOLDING_0.15_1024_512 tensor(4.7940, device='cuda:0')
nlls_wikitext_BIASING_0.25_1024_512 tensor(4.7940, device='cuda:0')
nlls_wikitext_TOP-K_0_2048_512 tensor(4.4980, device='cuda:0')
nlls_C4_TOP-K_0_2048_512 tensor(8.1308, device='cuda:0')
nlls_wikitext_THRESHOLDING_0.05_2048_512 tensor(4.5126, device='cuda:0')
nlls_C4_THRESHOLDING_0.05_2048_512 tensor(8.1528, device='cuda:0')
nlls_C4_THRESHOLDING_0.15_2048_512 tensor(8.3174, device='cuda:0')
nlls_C4_BIASING_0.25_2048_512 tensor(8.3920, device='cuda:0')
nlls_C4_THRESHOLDING_0.1_2048_512 tensor(8.2263, device='cuda:0')
nlls_C4_TOP-K_0_4096_512 tensor(8.0441, device='cuda:0')
nlls_C4_THRESHOLDING_0.05_4096_512 tensor(8.0622, device='cuda:0')
nlls_C4_THRESHOLDING_0.15_4096_512 tensor(8.2219, device='cuda:0')
nlls_C4_BIASING_0.25_4

In [2]:
import os
os.listdir()

['MMLU_perf_test.ipynb',
 '__pycache__',
 'mmlu',
 '.ipynb_checkpoints',
 'Mixtral-8x7B-Instruct-v0.1-offloading-demo',
 'MMLU_perf_test_topk.ipynb',
 'top-K_results',
 'Mixtral_profiling.ipynb',
 'Thresholding.ipynb',
 'categories.py',
 'demo-local.ipynb',
 'demo.ipynb',
 'mmlu.py',
 'thresholding_results',
 'MMLU_perf_test_thresholding.ipynb',
 'Mixtral-8x7B-Instruct-v0.1-offloading-demo2',
 'thresholding_0_025_results',
 'thresholding_0_075_results',
 'Mixtral-8x7B-Instruct-v0.1-offloading-demo1',
 'expert_frequencies.npy',
 'biasing_results',
 'MMLU_perf_test_biasing.ipynb',
 'top-K_6_offload_results',
 'thresholding_0_05_6E_Off_results',
 'thresholding_0_15_6E_Off_results',
 'MMLU_perf_test_thresholding-6_0075.ipynb',
 'Mixtral-8x7B-Instruct-v0.1-offloading-demo3',
 'MMLU_perf_test_thresholding-6_025.ipynb',
 'thresholding_0_25_6E_Off_results',
 'thresholding_0_5_6E_off_results',
 'nlls_wikitext_1024_512',
 'C4_PPL_calculations.py',
 'nlls_wikitext_TOP-K_0_1024_512',
 'nlls_wikite